<a href="https://colab.research.google.com/github/MarioSigal/Aprendizaje-Automatico-I-y-II/blob/main/TP_2_AA_RNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TP2 Aprendizaje Automatico
Archivo de los modelos basados en RNN

##IMPORTS & DRIVE


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
from typing import List, Dict, Any
import numpy as np
import pandas as pd
from google.colab import drive

In [ ]:
#Cargar drive
drive.mount('/content/drive')

##TOKENIZER & BERT EMBEDDINGS

In [ ]:
#cargamos bert
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
model_bert = BertModel.from_pretrained("bert-base-multilingual-cased")
model_bert.eval()

##DATASET

### ABRIMOS Y DAMOS FORMATO AL DATASET

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/colab/dataset_datos_total3.csv")

In [ ]:
# Agrupar tokens por párrafo
df_por_parrafos = df.groupby("instancia_id").agg({
    "token_id":list,
    "token": list,
    "punt_inicial": list,
    "punt_final": list,
    "capitalización": list
}).reset_index()


###CREAMOS CLASE DATASET DE PYTORCH

In [ ]:
from torch.utils.data import Dataset

class DynamicEmbeddingDataset(Dataset):
    """Dataset de pytorch que calcula embeddings dinámicamente"""
    def __init__(self, df, tokenizer, bert_model, modo="train"):
        self.df = df
        self.tokenizer = tokenizer
        self.bert_model = bert_model
        self.embedding_matrix = bert_model.embeddings.word_embeddings.weight
        self.embedding_matrix.requires_grad = False
        self.modo = modo  # "train" o "pred"

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        token_id_list = row["token_id"]

        # Calcular embeddings dinámicamente
        token_embeddings = []
        for token_id in token_id_list:
            if token_id is None or token_id == self.tokenizer.unk_token_id:
                token_id = self.tokenizer.unk_token_id
            emb = self.embedding_matrix[token_id].detach()
            token_embeddings.append(emb)

        # Convertir a tensor
        if token_embeddings:
            embeddings = torch.stack(token_embeddings)
        else:
            embeddings = torch.empty(0, self.embedding_matrix.shape[1])

        if self.modo == "pred": #para predecir, no devuelve los labels
            return embeddings, None

        # Preparar labels
        labels = {
            "punt_inicial": torch.tensor(row["punt_inicial"], dtype=torch.long),
            "punt_final": torch.tensor(row["punt_final"], dtype=torch.long),
            "capitalización": torch.tensor(row["capitalización"], dtype=torch.long)
        }

        return embeddings, labels

###FUNCIÓN DE PADDING

In [ ]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    """
    batch: lista de tuplas (embeddings, labels)
    """
    embeddings_list, labels_list = zip(*batch)

    # Pad embeddings (seq_len, embedding_dim) -> (batch_size, max_seq_len, embedding_dim)
    embeddings_padded = pad_sequence(embeddings_list, batch_first=True, padding_value=0.0)

    if labels_list[0] is None: # si estamos prediciendo, no tenemos labels
        return embeddings_padded, None

    # Pad labels
    punt_inicial = pad_sequence([l["punt_inicial"] for l in labels_list], batch_first=True, padding_value=-100).long()
    punt_final = pad_sequence([l["punt_final"] for l in labels_list], batch_first=True, padding_value=-100).long()
    capitalizacion = pad_sequence([l["capitalización"] for l in labels_list], batch_first=True, padding_value=-100).long()

    return embeddings_padded, {
        "punt_inicial": punt_inicial,
        "punt_final": punt_final,
        "capitalizacion": capitalizacion
    }

##ARQUITECTURAS

###RNN Unidireccional

In [ ]:
class RNN_Unidireccional(nn.Module):
    def __init__(self, embedding_dim = 768, hidden_dim1 = 128, hidden_dim2 = 32, num_layers= 2, dropout= 0.4):
        super(RNN_Unidireccional, self).__init__()

        # LSTM
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim1,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=False  # Unidireccional
            )

        # Dropout
        self.dropout = nn.Dropout(dropout)

        # Salidas
        self.punt_inicial_ff = nn.Sequential(
            nn.Linear(hidden_dim1, hidden_dim2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim2, 2)
            )
        self.punt_final_ff = nn.Sequential(
            nn.Linear(hidden_dim1, hidden_dim2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim2, 4)
            )
        self.capital_ff = nn.Sequential(
            nn.Linear(hidden_dim1, hidden_dim2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim2, 4)
            )

    def forward(self, embeddings):
        """
        embeddings: tensor de forma (batch_size, seq_len, embedding_dim)
        """
        # Pasamos los embeddings por la lstm
        outputs, _ = self.lstm(embeddings)

        # Les hacemos dropout a los outputs
        outputs = self.dropout(outputs)

        # Pasamos los outputs de la lstm por las salidas
        punt_inicial_logits = self.punt_inicial_ff(outputs)
        punt_final_logits = self.punt_final_ff(outputs)
        capital_logits = self.capital_ff(outputs)

        return {
            "puntuación inicial": punt_inicial_logits,
            "puntuación final": punt_final_logits,
            "capitalización": capital_logits,
        }


### RNN Bidireccional


In [ ]:
class RNN_Bidireccional(nn.Module):
    def __init__(self, embedding_dim = 768, hidden_dim1 = 64, hidden_dim2 = 32, num_layers= 2, dropout= 0.4): #reducimos hidden_dim1 para equiparar con la red unidireccional( 64*2 = 128)
        super(RNN_Bidireccional, self).__init__()

        # LSTM
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim1,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=True  # Bidireccional
            )

        lstm_output_dim = hidden_dim1 * 2 #128 igual que en la Unidireccinal

        # Dropout
        self.dropout = nn.Dropout(dropout)

        # Salidas
        self.punt_inicial_ff = nn.Sequential(
            nn.Linear(lstm_output_dim, hidden_dim2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim2, 2)
            )
        self.punt_final_ff = nn.Sequential(
            nn.Linear(lstm_output_dim, hidden_dim2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim2, 4)
            )
        self.capital_ff = nn.Sequential(
            nn.Linear(lstm_output_dim, hidden_dim2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim2, 4)
            )

    def forward(self, embeddings):
        """
        embeddings: tensor de forma (batch_size, seq_len, embedding_dim)
        """
        # Pasamos los embeddings por la lstm
        outputs, _ = self.lstm(embeddings)

        # Les hacemos dropout a los outputs
        outputs = self.dropout(outputs)

        # Pasamos los outputs de la lstm por las salidas
        punt_inicial_logits = self.punt_inicial_ff(outputs)
        punt_final_logits = self.punt_final_ff(outputs)
        capital_logits = self.capital_ff(outputs)

        return {
            "puntuación inicial": punt_inicial_logits,
            "puntuación final": punt_final_logits,
            "capitalización": capital_logits,
        }

##ENTRENAMIENTO DE LOS MODELOS

In [ ]:
from sklearn.model_selection import train_test_split
import torch.nn.utils as nn_utils

# Dividir en train/validation/test
train_df, temp = train_test_split(df_por_parrafos, test_size=0.2, shuffle=True, random_state=42)

val_df, test_df = train_test_split(temp, test_size=0.5, shuffle=True, random_state=42)

train_dataset = DynamicEmbeddingDataset(train_df.reset_index(drop=True), tokenizer, model_bert)
val_dataset = DynamicEmbeddingDataset(val_df.reset_index(drop=True), tokenizer, model_bert)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn)

###ENTRENAMIENTO UNIDIRECCIONAL
entrenamos por 30 epochs quedandonos con todos los modelos y viendo cual es el mejor segun como varia la loss, en train y val, epoch por epoch.

MEJOR MODELO: ÉPOCA 12

In [ ]:
from sklearn.model_selection import train_test_split
import torch.nn.utils as nn_utils

model = RNN_Unidireccional(embedding_dim=768, hidden_dim1=128, hidden_dim2=32,  num_layers=2, dropout=0.4)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Mover BERT a la misma device
model_bert = model_bert.to(device)

# Optimizador con weight decay (regularización L2)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.2, patience=3
)

# Pesos diferentes para cada tarea (opcional, ajustar según importancia)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100, label_smoothing=0.1)

# Gradient clipping
max_grad_norm = 1.0

num_epochs = 30
best_val_loss = float('inf')
patience_counter = 0
early_stop_patience = 30

for epoch in range(num_epochs):
    # ENTRENAMIENTO
    model.train()
    total_train_loss = 0
    total_loss_inicial = 0
    total_loss_final = 0
    total_loss_cap = 0

    for embeddings, labels in train_loader:
        embeddings = embeddings.to(device)
        labels = {k: v.to(device) for k, v in labels.items()}

        optimizer.zero_grad()
        outputs = model(embeddings)

        loss_inicial = 2*criterion(outputs["puntuación inicial"].permute(0,2,1), labels["punt_inicial"]) # le ponemos 2* a loss inicial para equiparar en magnitud con las demas
        loss_final = criterion(outputs["puntuación final"].permute(0,2,1), labels["punt_final"])
        loss_cap = criterion(outputs["capitalización"].permute(0,2,1), labels["capitalizacion"])

        # Pérdida total
        loss = loss_inicial + loss_final + loss_cap

        loss.backward()

        # Gradient clipping para evitar gradientes explosivos
        nn_utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        optimizer.step()

        total_train_loss += loss.item()
        total_loss_inicial += loss_inicial.item()
        total_loss_final += loss_final.item()
        total_loss_cap += loss_cap.item()

    avg_train_loss = total_train_loss / len(train_loader)

    # VALIDACIÓN
    model.eval()
    total_val_loss = 0
    val_loss_inicial = 0
    val_loss_final = 0
    val_loss_cap = 0

    with torch.no_grad():
        for embeddings, labels in val_loader:
            embeddings = embeddings.to(device)
            labels = {k: v.to(device) for k, v in labels.items()}

            outputs = model(embeddings)

            loss_inicial = 2*criterion(outputs["puntuación inicial"].permute(0,2,1), labels["punt_inicial"])  # le ponemos 2* a loss inicial para equiparar en magnitud con las demas

            loss_final = criterion(outputs["puntuación final"].permute(0,2,1), labels["punt_final"])

            loss_cap = criterion(outputs["capitalización"].permute(0,2,1), labels["capitalizacion"])

            loss = loss_inicial + loss_final + loss_cap

            total_val_loss += loss.item()
            val_loss_inicial += loss_inicial.item()
            val_loss_final += loss_final.item()
            val_loss_cap += loss_cap.item()

    avg_val_loss = total_val_loss / len(val_loader)

    # Actualizar learning rate
    scheduler.step(avg_val_loss)

    # Imprimir métricas detalladas
    print(f"Época {epoch+1}/{num_epochs}")
    print(f"  Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
    print(f"  Train - Inicial: {total_loss_inicial/len(train_loader):.4f}, Final: {total_loss_final/len(train_loader):.4f}, Cap: {total_loss_cap/len(train_loader):.4f}")
    print(f"  Val   - Inicial: {val_loss_inicial/len(val_loader):.4f}, Final: {val_loss_final/len(val_loader):.4f}, Cap: {val_loss_cap/len(val_loader):.4f}")
    print(f"  LR: {optimizer.param_groups[0]['lr']:.6f}")


    # Guardar cada epoca
    ruta_epoca = f"/content/drive/MyDrive/colab/modelosU/modelo_epoca_{epoch+1}.pt"
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'val_loss': avg_val_loss,
        }, ruta_epoca)


    # Guardar mejor modelo
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        ruta_mejor_modelo = "/content/drive/MyDrive/colab/modelosU/mejor_modelo.pt"
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': avg_val_loss,
        }, ruta_mejor_modelo)
        print(f"  ✓ Mejor modelo actualizado (Val Loss: {avg_val_loss:.4f})")
    else:
        patience_counter += 1
        print(f"  Sin mejora ({patience_counter}/{early_stop_patience})")

    # Early stopping
    if patience_counter >= early_stop_patience:
        print(f"\nEarly stopping en época {epoch+1}")
        break

    print("-" * 60)


print(f"\nEntrenamiento completado!")
print(f"Mejor modelo guardado en: {ruta_mejor_modelo}")
print(f"Mejor Val Loss: {best_val_loss:.4f}")

#Mostramos lo que printeo durante el entrenamiento

"""
Época 1/30
  Train Loss: 1.4731 | Val Loss: 1.4118
  Train - Inicial: 0.4391, Final: 0.5594, Cap: 0.4747
  Val   - Inicial: 0.4289, Final: 0.5407, Cap: 0.4423
  LR: 0.001000
  ✓ Mejor modelo actualizado (Val Loss: 1.4118)
------------------------------------------------------------
Época 2/30
  Train Loss: 1.4212 | Val Loss: 1.3945
  Train - Inicial: 0.4318, Final: 0.5423, Cap: 0.4470
  Val   - Inicial: 0.4282, Final: 0.5333, Cap: 0.4330
  LR: 0.001000
  ✓ Mejor modelo actualizado (Val Loss: 1.3945)
------------------------------------------------------------
Época 3/30
  Train Loss: 1.4088 | Val Loss: 1.3856
  Train - Inicial: 0.4308, Final: 0.5378, Cap: 0.4401
  Val   - Inicial: 0.4277, Final: 0.5289, Cap: 0.4290
  LR: 0.001000
  ✓ Mejor modelo actualizado (Val Loss: 1.3856)
------------------------------------------------------------
Época 4/30
  Train Loss: 1.4012 | Val Loss: 1.3827
  Train - Inicial: 0.4304, Final: 0.5351, Cap: 0.4358
  Val   - Inicial: 0.4276, Final: 0.5277, Cap: 0.4275
  LR: 0.001000
  ✓ Mejor modelo actualizado (Val Loss: 1.3827)
------------------------------------------------------------
Época 5/30
  Train Loss: 1.3952 | Val Loss: 1.3796
  Train - Inicial: 0.4301, Final: 0.5329, Cap: 0.4322
  Val   - Inicial: 0.4272, Final: 0.5269, Cap: 0.4255
  LR: 0.001000
  ✓ Mejor modelo actualizado (Val Loss: 1.3796)
------------------------------------------------------------
Época 6/30
  Train Loss: 1.3904 | Val Loss: 1.3784
  Train - Inicial: 0.4298, Final: 0.5315, Cap: 0.4291
  Val   - Inicial: 0.4273, Final: 0.5244, Cap: 0.4266
  LR: 0.001000
  ✓ Mejor modelo actualizado (Val Loss: 1.3784)
------------------------------------------------------------
Época 7/30
  Train Loss: 1.3865 | Val Loss: 1.3751
  Train - Inicial: 0.4295, Final: 0.5302, Cap: 0.4268
  Val   - Inicial: 0.4270, Final: 0.5240, Cap: 0.4241
  LR: 0.001000
  ✓ Mejor modelo actualizado (Val Loss: 1.3751)
------------------------------------------------------------
Época 8/30
  Train Loss: 1.3829 | Val Loss: 1.3750
  Train - Inicial: 0.4292, Final: 0.5292, Cap: 0.4245
  Val   - Inicial: 0.4270, Final: 0.5233, Cap: 0.4247
  LR: 0.001000
  ✓ Mejor modelo actualizado (Val Loss: 1.3750)
------------------------------------------------------------
Época 9/30
  Train Loss: 1.3799 | Val Loss: 1.3747
  Train - Inicial: 0.4290, Final: 0.5283, Cap: 0.4226
  Val   - Inicial: 0.4271, Final: 0.5231, Cap: 0.4246
  LR: 0.001000
  ✓ Mejor modelo actualizado (Val Loss: 1.3747)
------------------------------------------------------------
Época 10/30
  Train Loss: 1.3770 | Val Loss: 1.3750
  Train - Inicial: 0.4287, Final: 0.5274, Cap: 0.4208
  Val   - Inicial: 0.4272, Final: 0.5231, Cap: 0.4247
  LR: 0.001000
  Sin mejora (1/30)
------------------------------------------------------------
Época 11/30
  Train Loss: 1.3750 | Val Loss: 1.3759
  Train - Inicial: 0.4286, Final: 0.5268, Cap: 0.4197
  Val   - Inicial: 0.4272, Final: 0.5228, Cap: 0.4260
  LR: 0.001000
  Sin mejora (2/30)
------------------------------------------------------------
Época 12/30
  Train Loss: 1.3729 | Val Loss: 1.3747
  Train - Inicial: 0.4285, Final: 0.5262, Cap: 0.4182
  Val   - Inicial: 0.4271, Final: 0.5221, Cap: 0.4254
  LR: 0.001000
  ✓ Mejor modelo actualizado (Val Loss: 1.3747)
------------------------------------------------------------
Época 13/30
  Train Loss: 1.3708 | Val Loss: 1.3758
  Train - Inicial: 0.4282, Final: 0.5257, Cap: 0.4169
  Val   - Inicial: 0.4272, Final: 0.5222, Cap: 0.4265
  LR: 0.000200
  Sin mejora (1/30)
------------------------------------------------------------
Época 14/30
  Train Loss: 1.3635 | Val Loss: 1.3755
  Train - Inicial: 0.4275, Final: 0.5231, Cap: 0.4129
  Val   - Inicial: 0.4270, Final: 0.5218, Cap: 0.4266
  LR: 0.000200
  Sin mejora (2/30)
------------------------------------------------------------
Época 15/30
  Train Loss: 1.3618 | Val Loss: 1.3758
  Train - Inicial: 0.4273, Final: 0.5227, Cap: 0.4118
  Val   - Inicial: 0.4270, Final: 0.5216, Cap: 0.4272
  LR: 0.000200
  Sin mejora (3/30)
------------------------------------------------------------
Época 16/30
  Train Loss: 1.3604 | Val Loss: 1.3763
  Train - Inicial: 0.4272, Final: 0.5223, Cap: 0.4109
  Val   - Inicial: 0.4270, Final: 0.5215, Cap: 0.4277
  LR: 0.000200
  Sin mejora (4/30)
------------------------------------------------------------
Época 17/30
  Train Loss: 1.3595 | Val Loss: 1.3765
  Train - Inicial: 0.4271, Final: 0.5221, Cap: 0.4103
  Val   - Inicial: 0.4270, Final: 0.5217, Cap: 0.4278
  LR: 0.000040
  Sin mejora (5/30)
------------------------------------------------------------
Época 18/30
  Train Loss: 1.3576 | Val Loss: 1.3773
  Train - Inicial: 0.4269, Final: 0.5213, Cap: 0.4094
  Val   - Inicial: 0.4271, Final: 0.5216, Cap: 0.4286
  LR: 0.000040
  Sin mejora (6/30)
------------------------------------------------------------
Época 19/30
  Train Loss: 1.3571 | Val Loss: 1.3773
  Train - Inicial: 0.4268, Final: 0.5212, Cap: 0.4092
  Val   - Inicial: 0.4271, Final: 0.5216, Cap: 0.4286
  LR: 0.000040
  Sin mejora (7/30)
------------------------------------------------------------
Época 20/30
  Train Loss: 1.3567 | Val Loss: 1.3776
  Train - Inicial: 0.4267, Final: 0.5211, Cap: 0.4089
  Val   - Inicial: 0.4271, Final: 0.5217, Cap: 0.4288
  LR: 0.000040
  Sin mejora (8/30)
------------------------------------------------------------
Época 21/30
  Train Loss: 1.3570 | Val Loss: 1.3775
  Train - Inicial: 0.4268, Final: 0.5212, Cap: 0.4089
  Val   - Inicial: 0.4271, Final: 0.5217, Cap: 0.4287
  LR: 0.000008
  Sin mejora (9/30)
------------------------------------------------------------
Época 22/30
  Train Loss: 1.3564 | Val Loss: 1.3776
  Train - Inicial: 0.4268, Final: 0.5208, Cap: 0.4088
  Val   - Inicial: 0.4271, Final: 0.5216, Cap: 0.4289
  LR: 0.000008
  Sin mejora (10/30)
------------------------------------------------------------
Época 23/30
  Train Loss: 1.3562 | Val Loss: 1.3778
  Train - Inicial: 0.4266, Final: 0.5209, Cap: 0.4086
  Val   - Inicial: 0.4271, Final: 0.5216, Cap: 0.4291
  LR: 0.000008
  Sin mejora (11/30)
------------------------------------------------------------
Época 24/30
  Train Loss: 1.3562 | Val Loss: 1.3777
  Train - Inicial: 0.4266, Final: 0.5209, Cap: 0.4086
  Val   - Inicial: 0.4271, Final: 0.5217, Cap: 0.4290
  LR: 0.000008
  Sin mejora (12/30)
------------------------------------------------------------
Época 25/30
  Train Loss: 1.3560 | Val Loss: 1.3778
  Train - Inicial: 0.4266, Final: 0.5207, Cap: 0.4086
  Val   - Inicial: 0.4271, Final: 0.5216, Cap: 0.4290
  LR: 0.000002
  Sin mejora (13/30)
------------------------------------------------------------
Época 26/30
  Train Loss: 1.3561 | Val Loss: 1.3778
  Train - Inicial: 0.4266, Final: 0.5209, Cap: 0.4086
  Val   - Inicial: 0.4271, Final: 0.5216, Cap: 0.4290
  LR: 0.000002
  Sin mejora (14/30)
------------------------------------------------------------
Época 27/30
  Train Loss: 1.3561 | Val Loss: 1.3778
  Train - Inicial: 0.4267, Final: 0.5209, Cap: 0.4085
  Val   - Inicial: 0.4271, Final: 0.5216, Cap: 0.4290
  LR: 0.000002
  Sin mejora (15/30)
------------------------------------------------------------
Época 28/30
  Train Loss: 1.3562 | Val Loss: 1.3778
  Train - Inicial: 0.4267, Final: 0.5210, Cap: 0.4085
  Val   - Inicial: 0.4271, Final: 0.5216, Cap: 0.4291
  LR: 0.000002
  Sin mejora (16/30)
------------------------------------------------------------
Época 29/30
  Train Loss: 1.3560 | Val Loss: 1.3778
  Train - Inicial: 0.4266, Final: 0.5209, Cap: 0.4085
  Val   - Inicial: 0.4271, Final: 0.5216, Cap: 0.4291
  LR: 0.000000
  Sin mejora (17/30)
------------------------------------------------------------
Época 30/30
  Train Loss: 1.3561 | Val Loss: 1.3778
  Train - Inicial: 0.4266, Final: 0.5209, Cap: 0.4086
  Val   - Inicial: 0.4271, Final: 0.5216, Cap: 0.4291
  LR: 0.000000
  Sin mejora (18/30)
------------------------------------------------------------

Entrenamiento completado!
Mejor modelo guardado en: /content/drive/MyDrive/colab/modelosU/mejor_modelo.pt
Mejor Val Loss: 1.3747
"""

###ENTRENAMIENTO BIDIRECCIONAL
entrenamos por 30 epochs quedandonos con todos los modelos y viendo cual es el mejor segun como varia la loss, en train y val, epoch por epoch.

MEJOR MODELO: ÉPOCA 8

In [ ]:
model = RNN_Bidireccional(embedding_dim=768, hidden_dim1=64, hidden_dim2=32,  num_layers=2, dropout=0.4) # 64 para que 64*2 sea 128
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Mover BERT a la misma device
model_bert = model_bert.to(device)

# Optimizador con weight decay (regularización L2)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.2, patience=3
)

# Pesos diferentes para cada tarea (opcional, ajustar según importancia)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100, label_smoothing=0.1)

# Gradient clipping
max_grad_norm = 1.0

num_epochs = 30
best_val_loss = float('inf')
patience_counter = 0
early_stop_patience = 30

for epoch in range(num_epochs):
    # ENTRENAMIENTO
    model.train()
    total_train_loss = 0
    total_loss_inicial = 0
    total_loss_final = 0
    total_loss_cap = 0

    for embeddings, labels in train_loader:
        embeddings = embeddings.to(device)
        labels = {k: v.to(device) for k, v in labels.items()}

        optimizer.zero_grad()
        outputs = model(embeddings)

        loss_inicial = 2*criterion(outputs["puntuación inicial"].permute(0,2,1), labels["punt_inicial"]) # le ponemos 2* a loss inicial para equiparar en magnitud con las demas
        loss_final = criterion(outputs["puntuación final"].permute(0,2,1), labels["punt_final"])
        loss_cap = criterion(outputs["capitalización"].permute(0,2,1), labels["capitalizacion"])

        # Pérdida total
        loss = loss_inicial + loss_final + loss_cap

        loss.backward()

        # Gradient clipping para evitar gradientes explosivos
        nn_utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        optimizer.step()

        total_train_loss += loss.item()
        total_loss_inicial += loss_inicial.item()
        total_loss_final += loss_final.item()
        total_loss_cap += loss_cap.item()

    avg_train_loss = total_train_loss / len(train_loader)

    # VALIDACIÓN
    model.eval()
    total_val_loss = 0
    val_loss_inicial = 0
    val_loss_final = 0
    val_loss_cap = 0

    with torch.no_grad():
        for embeddings, labels in val_loader:
            embeddings = embeddings.to(device)
            labels = {k: v.to(device) for k, v in labels.items()}

            outputs = model(embeddings)

            loss_inicial = 2*criterion(outputs["puntuación inicial"].permute(0,2,1), labels["punt_inicial"])  # le ponemos 2* a loss inicial para equiparar en magnitud con las demas

            loss_final = criterion(outputs["puntuación final"].permute(0,2,1), labels["punt_final"])

            loss_cap = criterion(outputs["capitalización"].permute(0,2,1), labels["capitalizacion"])

            loss = loss_inicial + loss_final + loss_cap

            total_val_loss += loss.item()
            val_loss_inicial += loss_inicial.item()
            val_loss_final += loss_final.item()
            val_loss_cap += loss_cap.item()

    avg_val_loss = total_val_loss / len(val_loader)

    # Actualizar learning rate
    scheduler.step(avg_val_loss)

    # Imprimir métricas detalladas
    print(f"Época {epoch+1}/{num_epochs}")
    print(f"  Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
    print(f"  Train - Inicial: {total_loss_inicial/len(train_loader):.4f}, Final: {total_loss_final/len(train_loader):.4f}, Cap: {total_loss_cap/len(train_loader):.4f}")
    print(f"  Val   - Inicial: {val_loss_inicial/len(val_loader):.4f}, Final: {val_loss_final/len(val_loader):.4f}, Cap: {val_loss_cap/len(val_loader):.4f}")
    print(f"  LR: {optimizer.param_groups[0]['lr']:.6f}")


    # Guardar cada epoca
    ruta_epoca = f"/content/drive/MyDrive/colab/modelosB/modelo_epoca_{epoch+1}.pt"
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'val_loss': avg_val_loss,
        }, ruta_epoca)


    # Guardar mejor modelo
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        ruta_mejor_modelo = "/content/drive/MyDrive/colab/modelosB/mejor_modelo.pt"
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': avg_val_loss,
        }, ruta_mejor_modelo)
        print(f"  ✓ Mejor modelo actualizado (Val Loss: {avg_val_loss:.4f})")
    else:
        patience_counter += 1
        print(f"  Sin mejora ({patience_counter}/{early_stop_patience})")

    # Early stopping
    if patience_counter >= early_stop_patience:
        print(f"\nEarly stopping en época {epoch+1}")
        break

    print("-" * 60)


print(f"\nEntrenamiento completado!")
print(f"Mejor modelo guardado en: {ruta_mejor_modelo}")
print(f"Mejor Val Loss: {best_val_loss:.4f}")

#Mostramos lo que printeo durante el entrenamiento

"""
Época 1/30
  Train Loss: 1.3584 | Val Loss: 1.2807
  Train - Inicial: 0.4258, Final: 0.4811, Cap: 0.4515
  Val   - Inicial: 0.4152, Final: 0.4467, Cap: 0.4187
  LR: 0.001000
  ✓ Mejor modelo actualizado (Val Loss: 1.2807)
------------------------------------------------------------
Época 2/30
  Train Loss: 1.2907 | Val Loss: 1.2589
  Train - Inicial: 0.4165, Final: 0.4519, Cap: 0.4222
  Val   - Inicial: 0.4133, Final: 0.4370, Cap: 0.4085
  LR: 0.001000
  ✓ Mejor modelo actualizado (Val Loss: 1.2589)
------------------------------------------------------------
Época 3/30
  Train Loss: 1.2730 | Val Loss: 1.2504
  Train - Inicial: 0.4148, Final: 0.4442, Cap: 0.4140
  Val   - Inicial: 0.4126, Final: 0.4334, Cap: 0.4044
  LR: 0.001000
  ✓ Mejor modelo actualizado (Val Loss: 1.2504)
------------------------------------------------------------
Época 4/30
  Train Loss: 1.2615 | Val Loss: 1.2464
  Train - Inicial: 0.4136, Final: 0.4396, Cap: 0.4084
  Val   - Inicial: 0.4120, Final: 0.4315, Cap: 0.4029
  LR: 0.001000
  ✓ Mejor modelo actualizado (Val Loss: 1.2464)
------------------------------------------------------------
Época 5/30
  Train Loss: 1.2524 | Val Loss: 1.2430
  Train - Inicial: 0.4128, Final: 0.4360, Cap: 0.4036
  Val   - Inicial: 0.4117, Final: 0.4297, Cap: 0.4015
  LR: 0.001000
  ✓ Mejor modelo actualizado (Val Loss: 1.2430)
------------------------------------------------------------
Época 6/30
  Train Loss: 1.2451 | Val Loss: 1.2428
  Train - Inicial: 0.4120, Final: 0.4334, Cap: 0.3997
  Val   - Inicial: 0.4117, Final: 0.4293, Cap: 0.4019
  LR: 0.001000
  ✓ Mejor modelo actualizado (Val Loss: 1.2428)
------------------------------------------------------------
Época 7/30
  Train Loss: 1.2389 | Val Loss: 1.2419
  Train - Inicial: 0.4113, Final: 0.4311, Cap: 0.3965
  Val   - Inicial: 0.4115, Final: 0.4286, Cap: 0.4017
  LR: 0.001000
  ✓ Mejor modelo actualizado (Val Loss: 1.2419)
------------------------------------------------------------
Época 8/30
  Train Loss: 1.2340 | Val Loss: 1.2425
  Train - Inicial: 0.4107, Final: 0.4294, Cap: 0.3940
  Val   - Inicial: 0.4114, Final: 0.4285, Cap: 0.4026
  LR: 0.001000
  Sin mejora (1/30)
------------------------------------------------------------
Época 9/30
  Train Loss: 1.2293 | Val Loss: 1.2430
  Train - Inicial: 0.4101, Final: 0.4276, Cap: 0.3917
  Val   - Inicial: 0.4116, Final: 0.4287, Cap: 0.4026
  LR: 0.001000
  Sin mejora (2/30)
------------------------------------------------------------
Época 10/30
  Train Loss: 1.2254 | Val Loss: 1.2444
  Train - Inicial: 0.4096, Final: 0.4260, Cap: 0.3898
  Val   - Inicial: 0.4118, Final: 0.4295, Cap: 0.4031
  LR: 0.001000
  Sin mejora (3/30)
------------------------------------------------------------
Época 11/30
  Train Loss: 1.2221 | Val Loss: 1.2438
  Train - Inicial: 0.4092, Final: 0.4247, Cap: 0.3883
  Val   - Inicial: 0.4118, Final: 0.4289, Cap: 0.4031
  LR: 0.000200
  Sin mejora (4/30)
------------------------------------------------------------
Época 12/30
  Train Loss: 1.2103 | Val Loss: 1.2445
  Train - Inicial: 0.4075, Final: 0.4195, Cap: 0.3832
  Val   - Inicial: 0.4119, Final: 0.4292, Cap: 0.4034
  LR: 0.000200
  Sin mejora (5/30)
------------------------------------------------------------
Época 13/30
  Train Loss: 1.2064 | Val Loss: 1.2456
  Train - Inicial: 0.4070, Final: 0.4178, Cap: 0.3816
  Val   - Inicial: 0.4122, Final: 0.4294, Cap: 0.4040
  LR: 0.000200
  Sin mejora (6/30)
------------------------------------------------------------
Época 14/30
  Train Loss: 1.2045 | Val Loss: 1.2464
  Train - Inicial: 0.4067, Final: 0.4170, Cap: 0.3808
  Val   - Inicial: 0.4123, Final: 0.4295, Cap: 0.4046
  LR: 0.000200
  Sin mejora (7/30)
------------------------------------------------------------
Época 15/30
  Train Loss: 1.2027 | Val Loss: 1.2470
  Train - Inicial: 0.4065, Final: 0.4161, Cap: 0.3801
  Val   - Inicial: 0.4122, Final: 0.4299, Cap: 0.4048
  LR: 0.000040
  Sin mejora (8/30)
------------------------------------------------------------
Época 16/30
  Train Loss: 1.1998 | Val Loss: 1.2487
  Train - Inicial: 0.4061, Final: 0.4147, Cap: 0.3791
  Val   - Inicial: 0.4128, Final: 0.4303, Cap: 0.4055
  LR: 0.000040
  Sin mejora (9/30)
------------------------------------------------------------
Época 17/30
  Train Loss: 1.1990 | Val Loss: 1.2490
  Train - Inicial: 0.4060, Final: 0.4143, Cap: 0.3788
  Val   - Inicial: 0.4126, Final: 0.4304, Cap: 0.4060
  LR: 0.000040
  Sin mejora (10/30)
------------------------------------------------------------
Época 18/30
  Train Loss: 1.1983 | Val Loss: 1.2492
  Train - Inicial: 0.4058, Final: 0.4141, Cap: 0.3784
  Val   - Inicial: 0.4127, Final: 0.4304, Cap: 0.4061
  LR: 0.000040
  Sin mejora (11/30)
------------------------------------------------------------
Época 19/30
  Train Loss: 1.1979 | Val Loss: 1.2498
  Train - Inicial: 0.4057, Final: 0.4139, Cap: 0.3783
  Val   - Inicial: 0.4127, Final: 0.4308, Cap: 0.4063
  LR: 0.000008
  Sin mejora (12/30)
------------------------------------------------------------
Época 20/30
  Train Loss: 1.1975 | Val Loss: 1.2496
  Train - Inicial: 0.4056, Final: 0.4137, Cap: 0.3782
  Val   - Inicial: 0.4127, Final: 0.4306, Cap: 0.4063
  LR: 0.000008
  Sin mejora (13/30)
------------------------------------------------------------
Época 21/30
  Train Loss: 1.1973 | Val Loss: 1.2495
  Train - Inicial: 0.4057, Final: 0.4136, Cap: 0.3781
  Val   - Inicial: 0.4128, Final: 0.4306, Cap: 0.4062
  LR: 0.000008
  Sin mejora (14/30)
------------------------------------------------------------
Época 22/30
  Train Loss: 1.1970 | Val Loss: 1.2499
  Train - Inicial: 0.4057, Final: 0.4134, Cap: 0.3779
  Val   - Inicial: 0.4128, Final: 0.4307, Cap: 0.4064
  LR: 0.000008
  Sin mejora (15/30)
------------------------------------------------------------
Época 23/30
  Train Loss: 1.1972 | Val Loss: 1.2499
  Train - Inicial: 0.4057, Final: 0.4135, Cap: 0.3779
  Val   - Inicial: 0.4128, Final: 0.4307, Cap: 0.4065
  LR: 0.000002
  Sin mejora (16/30)
------------------------------------------------------------
Época 24/30
  Train Loss: 1.1971 | Val Loss: 1.2499
  Train - Inicial: 0.4057, Final: 0.4135, Cap: 0.3779
  Val   - Inicial: 0.4127, Final: 0.4307, Cap: 0.4064
  LR: 0.000002
  Sin mejora (17/30)
------------------------------------------------------------
Época 25/30
  Train Loss: 1.1970 | Val Loss: 1.2500
  Train - Inicial: 0.4056, Final: 0.4134, Cap: 0.3779
  Val   - Inicial: 0.4128, Final: 0.4308, Cap: 0.4064
  LR: 0.000002
  Sin mejora (18/30)
------------------------------------------------------------
Época 26/30
  Train Loss: 1.1969 | Val Loss: 1.2500
  Train - Inicial: 0.4056, Final: 0.4134, Cap: 0.3779
  Val   - Inicial: 0.4128, Final: 0.4308, Cap: 0.4065
  LR: 0.000002
  Sin mejora (19/30)
------------------------------------------------------------
Época 27/30
  Train Loss: 1.1969 | Val Loss: 1.2500
  Train - Inicial: 0.4057, Final: 0.4134, Cap: 0.3779
  Val   - Inicial: 0.4128, Final: 0.4308, Cap: 0.4064
  LR: 0.000000
  Sin mejora (20/30)
------------------------------------------------------------
Época 28/30
  Train Loss: 1.1968 | Val Loss: 1.2500
  Train - Inicial: 0.4056, Final: 0.4134, Cap: 0.3778
  Val   - Inicial: 0.4128, Final: 0.4308, Cap: 0.4065
  LR: 0.000000
  Sin mejora (21/30)
------------------------------------------------------------
Época 29/30
  Train Loss: 1.1969 | Val Loss: 1.2500
  Train - Inicial: 0.4056, Final: 0.4134, Cap: 0.3779
  Val   - Inicial: 0.4128, Final: 0.4308, Cap: 0.4065
  LR: 0.000000
  Sin mejora (22/30)
------------------------------------------------------------
Época 30/30
  Train Loss: 1.1968 | Val Loss: 1.2500
  Train - Inicial: 0.4056, Final: 0.4134, Cap: 0.3778
  Val   - Inicial: 0.4128, Final: 0.4308, Cap: 0.4065
  LR: 0.000000
  Sin mejora (23/30)
------------------------------------------------------------

Entrenamiento completado!
Mejor modelo guardado en: /content/drive/MyDrive/colab/modelosB/mejor_modelo.pt
Mejor Val Loss: 1.2419
"""

##EVALUACIÓN DE MODELOS

###EVALUACIÓN UNIDIRECCIONAL

In [ ]:
from sklearn.metrics import f1_score

model = RNN_Unidireccional(embedding_dim=768, hidden_dim1=128, hidden_dim2=32,  num_layers=2, dropout=0.4)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# ---------------------------------------
# EVALUACIÓN SOBRE EL TEST SET
# ---------------------------------------

test_dataset = DynamicEmbeddingDataset(test_df.reset_index(drop=True), tokenizer, model_bert)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn)

# Cargar el mejor modelo
checkpoint = torch.load("/content/drive/MyDrive/colab/modelosU/mejor_modelo.pt") #modelosU para unidireccional
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print("\n=== Evaluando en TEST SET con mejor modelo Unidireccional ===\n")

criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)

total_test_loss = 0
loss_init = 0
loss_final = 0
loss_cap = 0

# Para F1
all_true_init  = []
all_pred_init  = []

all_true_final = []
all_pred_final = []

all_true_cap   = []
all_pred_cap   = []

correct_init = 0
correct_final = 0
correct_cap = 0

total_tokens = 0

with torch.no_grad():
    for embeddings, labels in test_loader:
        embeddings = embeddings.to(device)
        labels = {k: v.to(device) for k, v in labels.items()}

        outputs = model(embeddings)

        # Loss
        l_i = 2*criterion(outputs["puntuación inicial"].permute(0,2,1), labels["punt_inicial"]) # 2* para equipararla con las otras
        l_f = criterion(outputs["puntuación final"].permute(0,2,1), labels["punt_final"])
        l_c = criterion(outputs["capitalización"].permute(0,2,1), labels["capitalizacion"])

        loss = l_i + l_f + l_c

        total_test_loss += loss.item()
        loss_init += l_i.item()
        loss_final += l_f.item()
        loss_cap += l_c.item()

        # Predicciones (batch, seq)
        pred_init = outputs["puntuación inicial"].argmax(dim=-1)
        pred_final = outputs["puntuación final"].argmax(dim=-1)
        pred_cap = outputs["capitalización"].argmax(dim=-1)

        mask = (labels["punt_inicial"] != -100)

        # Guardar para F1
        all_true_init.extend(labels["punt_inicial"][mask].cpu().tolist())
        all_pred_init.extend(pred_init[mask].cpu().tolist())

        all_true_final.extend(labels["punt_final"][mask].cpu().tolist())
        all_pred_final.extend(pred_final[mask].cpu().tolist())

        all_true_cap.extend(labels["capitalizacion"][mask].cpu().tolist())
        all_pred_cap.extend(pred_cap[mask].cpu().tolist())

        # Accuracy por tarea
        # PREDICCIONES → (batch, seq, clases)
        pred_init = outputs["puntuación inicial"].argmax(dim=-1)
        pred_final = outputs["puntuación final"].argmax(dim=-1)
        pred_cap = outputs["capitalización"].argmax(dim=-1)

        mask = (labels["punt_inicial"] != -100)

        correct_init += ((pred_init == labels["punt_inicial"]) * mask).sum().item()
        correct_final += ((pred_final == labels["punt_final"]) * mask).sum().item()
        correct_cap += ((pred_cap == labels["capitalizacion"]) * mask).sum().item()

        total_tokens += mask.sum().item()


# ----------------------------
#   CÁLCULO DE F1 MACRO
# ----------------------------

# Puntuación inicial: 2 clases → ¿, ""
f1_init = f1_score(all_true_init, all_pred_init, average="macro")

# Puntuación final: 4 clases → ,  .  ?  ""
f1_final = f1_score(all_true_final, all_pred_final, average="macro")

# Capitalización: 4 clases → 0 1 2 3
f1_cap = f1_score(all_true_cap, all_pred_cap, average="macro")


# ----------------------------
#     PRINT RESULTADOS
# ----------------------------
print("\n===== RESULTADOS TEST =====")
print(f"Test Loss Total: {total_test_loss / len(test_loader):.4f}")
print(f"  Inicial: {loss_init / len(test_loader):.4f}")
print(f"  Final  : {loss_final / len(test_loader):.4f}")
print(f"  Capital: {loss_cap / len(test_loader):.4f}")

print("\n===== F1 MACRO =====")
print(f"  Puntuación inicial (¿, \"\"): {f1_init:.4f}")
print(f"  Puntuación final   (,, ., ?, \"\"): {f1_final:.4f}")
print(f"  Capitalización     (0,1,2,3): {f1_cap:.4f}")

print("\n===== ACCURACY =====")
print(f" Inicial: {correct_init / total_tokens:.4f}")
print(f" Final : {correct_final / total_tokens:.4f}")
print(f" Capital: {correct_cap / total_tokens:.4f}")

# Mostramos lo que printeo
"""
=== Evaluando en TEST SET con mejor modelo Unidireccional ===


===== RESULTADOS TEST =====
Test Loss Total: 0.6204
  Inicial: 0.1444
  Final  : 0.3020
  Capital: 0.1741

===== F1 MACRO =====
  Puntuación inicial (¿, ""): 0.7869
  Puntuación final   (,, ., ?, ""): 0.3820
  Capitalización     (0,1,2,3): 0.7420

===== ACCURACY =====
 Inicial: 0.9920
 Final : 0.9171
 Capital: 0.9658
"""

###EVALUACIÓN BIDIRECCIONAL




In [ ]:
from sklearn.metrics import f1_score

model = RNN_Bidireccional(embedding_dim=768, hidden_dim1=128, hidden_dim2=32,  num_layers=2, dropout=0.4)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# ---------------------------------------
# EVALUACIÓN SOBRE EL TEST SET
# ---------------------------------------

test_dataset = DynamicEmbeddingDataset(test_df.reset_index(drop=True), tokenizer, model_bert)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn)

# Cargar el mejor modelo
checkpoint = torch.load("/content/drive/MyDrive/colab/modelosB/mejor_modelo.pt") #modelosB para bidireccional
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print("\n=== Evaluando en TEST SET con mejor modelo Bidireccional ===\n")

criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)

total_test_loss = 0
loss_init = 0
loss_final = 0
loss_cap = 0

# Para F1
all_true_init  = []
all_pred_init  = []

all_true_final = []
all_pred_final = []

all_true_cap   = []
all_pred_cap   = []

correct_init = 0
correct_final = 0
correct_cap = 0

total_tokens = 0

with torch.no_grad():
    for embeddings, labels in test_loader:
        embeddings = embeddings.to(device)
        labels = {k: v.to(device) for k, v in labels.items()}

        outputs = model(embeddings)

        # Loss
        l_i = 2*criterion(outputs["puntuación inicial"].permute(0,2,1), labels["punt_inicial"]) # 2* para equipararla con las otras
        l_f = criterion(outputs["puntuación final"].permute(0,2,1), labels["punt_final"])
        l_c = criterion(outputs["capitalización"].permute(0,2,1), labels["capitalizacion"])

        loss = l_i + l_f + l_c

        total_test_loss += loss.item()
        loss_init += l_i.item()
        loss_final += l_f.item()
        loss_cap += l_c.item()

        # Predicciones (batch, seq)
        pred_init = outputs["puntuación inicial"].argmax(dim=-1)
        pred_final = outputs["puntuación final"].argmax(dim=-1)
        pred_cap = outputs["capitalización"].argmax(dim=-1)

        mask = (labels["punt_inicial"] != -100)

        # Guardar para F1
        all_true_init.extend(labels["punt_inicial"][mask].cpu().tolist())
        all_pred_init.extend(pred_init[mask].cpu().tolist())

        all_true_final.extend(labels["punt_final"][mask].cpu().tolist())
        all_pred_final.extend(pred_final[mask].cpu().tolist())

        all_true_cap.extend(labels["capitalizacion"][mask].cpu().tolist())
        all_pred_cap.extend(pred_cap[mask].cpu().tolist())

        # Accuracy por tarea
        # PREDICCIONES → (batch, seq, clases)
        pred_init = outputs["puntuación inicial"].argmax(dim=-1)
        pred_final = outputs["puntuación final"].argmax(dim=-1)
        pred_cap = outputs["capitalización"].argmax(dim=-1)

        mask = (labels["punt_inicial"] != -100)

        correct_init += ((pred_init == labels["punt_inicial"]) * mask).sum().item()
        correct_final += ((pred_final == labels["punt_final"]) * mask).sum().item()
        correct_cap += ((pred_cap == labels["capitalizacion"]) * mask).sum().item()

        total_tokens += mask.sum().item()


# ----------------------------
#   CÁLCULO DE F1 MACRO
# ----------------------------

# Puntuación inicial: 2 clases → ¿, ""
f1_init = f1_score(all_true_init, all_pred_init, average="macro")

# Puntuación final: 4 clases → ,  .  ?  ""
f1_final = f1_score(all_true_final, all_pred_final, average="macro")

# Capitalización: 4 clases → 0 1 2 3
f1_cap = f1_score(all_true_cap, all_pred_cap, average="macro")


# ----------------------------
#     PRINT RESULTADOS
# ----------------------------
print("\n===== RESULTADOS TEST =====")
print(f"Test Loss Total: {total_test_loss / len(test_loader):.4f}")
print(f"  Inicial: {loss_init / len(test_loader):.4f}")
print(f"  Final  : {loss_final / len(test_loader):.4f}")
print(f"  Capital: {loss_cap / len(test_loader):.4f}")

print("\n===== F1 MACRO =====")
print(f"  Puntuación inicial (¿, \"\"): {f1_init:.4f}")
print(f"  Puntuación final   (,, ., ?, \"\"): {f1_final:.4f}")
print(f"  Capitalización     (0,1,2,3): {f1_cap:.4f}")

print("\n===== ACCURACY =====")
print(f" Inicial: {correct_init / total_tokens:.4f}")
print(f" Final : {correct_final / total_tokens:.4f}")
print(f" Capital: {correct_cap / total_tokens:.4f}")

# Mostramos lo que printeo
"""
=== Evaluando en TEST SET con mejor modelo Bidireccional ===


===== RESULTADOS TEST =====
Test Loss Total: 0.4457
  Inicial: 0.1223
  Final  : 0.1812
  Capital: 0.1423

===== F1 MACRO =====
  Puntuación inicial (¿, ""): 0.9208
  Puntuación final   (,, ., ?, ""): 0.7996
  Capitalización     (0,1,2,3): 0.8125

===== ACCURACY =====
 Inicial: 0.9963
 Final : 0.9574
 Capital: 0.9762
"""


##PREDICCIONES CON EL MEJOR MODELO (BIDIRECCIONAL)

In [ ]:
# Mapas para convertir clases -> símbolos
map_punt_inicial = {
    0: "",
    1: "¿"
}

map_punt_final = {
    0: "",
    1: ".",
    2: ",",
    3: "?"
}

model = RNN_Bidireccional(embedding_dim=768, hidden_dim1=128, hidden_dim2=32, num_layers=2, dropout=0.4)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Cargar mejor modelo
checkpoint = torch.load("/content/drive/MyDrive/colab/modelosB/mejor_modelo.pt")
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

df_test = pd.read_csv("/content/drive/MyDrive/datos_test.csv")

# Agrupar tokens por párrafo
test_df = df_test.groupby("instancia_id").agg({
    "token_id": list,
    "token": list,
    "punt_inicial": list,
    "punt_final": list,
    "capitalización": list
}).reset_index()

pred_dataset = DynamicEmbeddingDataset(test_df.reset_index(drop=True), tokenizer, model_bert, modo="pred")
pred_loader = DataLoader(pred_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn)

pred_inicial = []
pred_final = []
pred_cap = []

with torch.no_grad():
    global_index = 0  # para trackear la fila correcta en test_df

    for embeddings, _ in pred_loader:
        embeddings = embeddings.to(device)
        outputs = model(embeddings)

        pred_init  = outputs["puntuación inicial"].argmax(dim=-1).cpu()
        pred_fin   = outputs["puntuación final"].argmax(dim=-1).cpu()
        pred_capit = outputs["capitalización"].argmax(dim=-1).cpu()

        batch_size = pred_init.shape[0]

        for i in range(batch_size):
            real_len = len(test_df.loc[global_index, "token"])

            pred_init_row  = [map_punt_inicial[int(x)] for x in pred_init[i][:real_len]]
            pred_fin_row   = [map_punt_final[int(x)]   for x in pred_fin[i][:real_len]]
            pred_cap_row   = pred_capit[i][:real_len].tolist()

            pred_inicial.append(pred_init_row)
            pred_final.append(pred_fin_row)
            pred_cap.append(pred_cap_row)

            global_index += 1


# Expandir listas de listas
df_test["punt_inicial"]   = [p for par in pred_inicial for p in par]
df_test["punt_final"]     = [p for par in pred_final   for p in par]
df_test["capitalización"] = [p for par in pred_cap     for p in par]

ruta_salida = "/content/drive/MyDrive/colab/predicciones_test.csv"
df_test.to_csv(ruta_salida, index=False)

print(f"\nPredicciones generadas correctamente y guardadas en:\n{ruta_salida}")


##OTROS

In [ ]:
def predecir_parrafo(texto, modelo, tokenizer, bert_model, device):
    """
    Recibe un párrafo de texto y devuelve las predicciones de puntuación y capitalización.

    Args:
        texto: string con el texto a procesar
        modelo: modelo entrenado
        tokenizer: tokenizer de BERT
        bert_model: modelo BERT para embeddings
        device: 'cuda' o 'cpu'

    Returns:
        dict con tokens y sus predicciones
    """
    modelo.eval()

    # Tokenizar el texto
    tokens = tokenizer.tokenize(texto)

    # Obtener embeddings
    token_embeddings = []
    for token in tokens:
        token_id = tokenizer.convert_tokens_to_ids(token)
        if token_id is None or token_id == tokenizer.unk_token_id:
            token_id = tokenizer.unk_token_id
        emb = bert_model.embeddings.word_embeddings.weight[token_id].detach()
        token_embeddings.append(emb)

    # Convertir a tensor y añadir dimensión de batch
    embeddings = torch.stack(token_embeddings).unsqueeze(0).to(device)  # (1, seq_len, 768)

    # Hacer predicción
    with torch.no_grad():
        outputs = modelo(embeddings)

    # Obtener las clases predichas
    punt_inicial_pred = torch.argmax(outputs["puntuación inicial"], dim=-1).squeeze().cpu().numpy()
    punt_final_pred = torch.argmax(outputs["puntuación final"], dim=-1).squeeze().cpu().numpy()
    capital_pred = torch.argmax(outputs["capitalización"], dim=-1).squeeze().cpu().numpy()

    # Mapeos de clases a etiquetas legibles
    punt_inicial_map = {0: "Sin puntuación", 1: "Con puntuación"}
    punt_final_map = {0: "Ninguna", 1: "Punto", 2: "Coma", 3: "Otro"}
    capital_map = {0: "Minúscula", 1: "Primera mayúscula", 2: "Todo mayúsculas", 3: "Otro"}

    # Crear resultado
    resultados = []
    for i, token in enumerate(tokens):
        resultados.append({
            "token": token,
            "puntuación_inicial": punt_inicial_map.get(int(punt_inicial_pred[i]), "Desconocido"),
            "puntuación_final": punt_final_map.get(int(punt_final_pred[i]), "Desconocido"),
            "capitalización": capital_map.get(int(capital_pred[i]), "Desconocido")
        })

    return resultados

def reconstruir_texto(resultados):
    """
    Reconstruye el texto con puntuación y capitalización a partir de las predicciones.
    """
    texto_reconstruido = ""

    for resultado in resultados:
        token = resultado["token"]

        # Aplicar capitalización
        if resultado["capitalización"] == "Primera mayúscula":
            token = token.capitalize()
        elif resultado["capitalización"] == "Todo mayúsculas":
            token = token.upper()

        # Añadir puntuación inicial
        if resultado["puntuación_inicial"] == "Con puntuación":
            # Aquí podrías decidir qué puntuación añadir (ej. ¿, ¡, etc.)
            pass

        # Añadir el token
        if token.startswith("##"):
            texto_reconstruido += token[2:]
        else:
            if texto_reconstruido:
                texto_reconstruido += " " + token
            else:
                texto_reconstruido += token

        # Añadir puntuación final
        if resultado["puntuación_final"] == "Punto":
            texto_reconstruido += "."
        elif resultado["puntuación_final"] == "Coma":
            texto_reconstruido += ","

    return texto_reconstruido

In [ ]:

# Cargar el mejor modelo
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Cargando el mejor modelo...")
checkpoint = torch.load("/content/drive/MyDrive/modelos/modelo_final_30e.pt", map_location=device)
modelo_cargado = ModeloUnidireccional(embedding_dim=768, hidden_dim=256, num_layers=2, dropout=0.3)
modelo_cargado.load_state_dict(checkpoint['model_state_dict'])
modelo_cargado = modelo_cargado.to(device)
print(f"Modelo cargado (época {checkpoint['epoch']}, val_loss: {checkpoint['val_loss']:.4f})")

# Ejemplo de uso
texto_ejemplo = "había perdido la esperanza pero seguía creyendo porque sabía que era un tenista demasiado bueno para no ganar también aquí pero no me he alegrado cuando  ha perdido lo respeto demasiado estabas nervioso antes de la final"
print(f"\nTexto de entrada: '{texto_ejemplo}'\n")

predicciones = predecir_parrafo(texto_ejemplo, modelo_cargado, tokenizer, model_bert, device)

# Mostrar predicciones detalladas
print("Predicciones por token:")
print("-" * 80)
for pred in predicciones:
    print(f"Token: {pred['token']:15s} | Punt. Inicial: {pred['puntuación_inicial']:20s} | "
          f"Punt. Final: {pred['puntuación_final']:10s} | Cap: {pred['capitalización']}")

# Reconstruir texto
texto_reconstruido = reconstruir_texto(predicciones)
print("\n" + "=" * 80)
print(f"Texto reconstruido: '{texto_reconstruido}'")
print("=" * 80)

In [ ]:
def reconstruccion_texto(dataSet): #data set con columnas: instancia_id (cte), token_id, token, capitalización, punt_inicial, punt_final
  frase = ''
  frase_final = ''
  n = dataSet.shape[0]
  i = 0
  capitalizaciones = []
  punt_finales = []
  punt_iniciales = []

  for fila in dataSet.iterrows():
    subpalabra = fila[1]['token']

    if subpalabra[:2] == '##':
      subpalabra = subpalabra[2:]
    else:
      capitalizaciones.append(fila[1]['capitalización'])
      punt_iniciales.append(fila[1]['punt_inicial'])

    if i == n-1:
      frase += subpalabra
      punt_finales.append(fila[1]['punt_final'])
    else:
      if dataSet.iloc[i+1]['token'][:2] == '##':
        frase += subpalabra
      else:
        frase += subpalabra + ' '
        punt_finales.append(fila[1]['punt_final'])
    i +=1

  palabras = frase.split()
  for i, palabra in enumerate(palabras):

    if punt_iniciales[i] == 1:
      frase_final += '¿'

    if capitalizaciones[i] == 0:
      frase_final += palabra
    elif capitalizaciones[i] == 1:
      frase_final += palabra.capitalize()
    elif capitalizaciones[i] == 3:
      frase_final += palabra.upper()
    else:
      frase_final += palabra[:-1].capitalize() + palabra[-1].upper()

    if punt_finales[i] == 1:
      frase_final += '.'
    elif punt_finales[i] == 2:
      frase_final += ','
    elif punt_finales[i] == 3:
      frase_final += '?'



    if i != len(palabras) - 1 and palabras[i+1] != "'" and palabras[i] != "'":
        frase_final += ' '

  return frase_final

In [ ]:
for i in range(1,100):
  print(reconstruccion_texto(df_test[df_test["instancia_id"]==i]))